## Request
---

I am interested in all of the Zillow APIs but GetComps and GetDeepComps are of most interest. There are a few parameters that I would like to find via the APIs:
 
* Property type
* Number of Units
* Price
* Zstimate Price
* Year Built
* Property tax
* Rental income (if available)
* School district rating
* Crime Rate
* HOA 
* Etc. (only because we will tweak our property searching criteria over time)
 
The output from those variables could be gathered in a dict or data frame for further analysis. One measure is the cap rate. The calculation is here:
https://www.propertymetrics.com/blog/2013/06/03/cap-rate/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import configparser
import requests as r
from xml.etree import ElementTree as ET

In [3]:
# Get API key for Zillow from config file
config = configparser.ConfigParser()
config.read('../config.ini')
zws_id = config['api_key']['zillow']

# Or plug in own by commenting out above and comment in below
# zws_id = "_________________"

> Example Property
 https://www.zillow.com/b/toscana-irvine-ca-5XpMKb/

## Example Rental Apartment Property
---
https://www.zillow.com/b/toscana-irvine-ca-5XpMKb/

In [19]:
# GetSearchResults API
# http://www.zillow.com/webservice/GetSearchResults.htm?zws-id=<ZWSID>&address=35+Via+Lucca&citystatezip=92612

address = '35+Via+Lucca'
citystatezip = '92612'

searchURL = 'http://www.zillow.com/webservice/GetSearchResults.htm?' \
           + 'zws-id=' + zws_id \
           + '&address=' + address \
           + '&citystatezip=' + citystatezip

get_search = r.get(searchURL)
root_search = ET.fromstring(get_search.content)

In [20]:
for child in root_search.iter('*'):
    print(child.tag, child.text, child.attrib)

{http://www.zillow.com/static/xsd/SearchResults.xsd}searchresults None {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/b20c067/static/xsd/SearchResults.xsd'}
request None {}
address 35 Via Lucca {}
citystatezip 92612 {}
message None {}
text Request successfully processed {}
code 0 {}
response None {}
results None {}
result None {}
zpid 2088377383 {}
links None {}
homedetails https://www.zillow.com/homedetails/35-Via-Lucca-H320-Irvine-CA-92612/2088377383_zpid/ {}
mapthishome http://www.zillow.com/homes/2088377383_zpid/ {}
comparables http://www.zillow.com/homes/comps/2088377383_zpid/ {}
address None {}
street 35 Via Lucca # H320 {}
zipcode 92612 {}
city Irvine {}
state CA {}
latitude 33.666722 {}
longitude -117.849519 {}
zestimate None {}
amount None {'currency': 'USD'}
last-updated 01/01/1970 {}
oneWeekChange None {'deprecated': 'true'}
valueChange None {}
valuationRange None {}
low N

In [27]:
unit_list = []

for child in root_search.iter('zpid'):
    unit_list.append(child.text)

print('Number of Units: {}'.format(len(unit_list)))

Number of Units: 17


In [29]:
unit_list

['2088377383',
 '2089501384',
 '2089220042',
 '2088894079',
 '2088630946',
 '2088526413',
 '2088342578',
 '2088476057',
 '2088954975',
 '2088989606',
 '2088624331',
 '2089361942',
 '2089501395',
 '2088675627',
 '2088528060',
 '2089501371',
 '2089501372']

> Above is the list of the units available in this complex.
* The below makes a function of the process above for repeatability on other properties.

In [13]:
def get_search_api(address, citystatezip):
    
    searchURL = 'http://www.zillow.com/webservice/GetSearchResults.htm?' \
           + 'zws-id=' + zws_id + '&address=' + address + '&citystatezip=' + citystatezip + '&rentzestimate=true'

    get_search = r.get(searchURL)
    root_search = ET.fromstring(get_search.content)
    
    unit_list = []

    for child in root_search.iter('zpid'):
        unit_list.append(child.text)

    print('Number of Units Available: {}'.format(len(unit_list)))
    
    for child in root_search.iter('*'):
        print(child.tag, child.text, child.attrib)
    
    return unit_list

In [14]:
# GetSearchResults API
# https://www.zillow.com/b/park-place-apartment-homes-irvine-ca-5h5YZg/
park_place_apt_irvine = get_search_api('3395+Michelson+Dr', '92612')
park_place_apt_irvine

Number of Units Available: 1
{http://www.zillow.com/static/xsd/SearchResults.xsd}searchresults None {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/b20c067/static/xsd/SearchResults.xsd'}
request None {}
address 3395 Michelson Dr {}
citystatezip 92612 {}
message None {}
text Request successfully processed {}
code 0 {}
response None {}
results None {}
result None {}
zpid 122244533 {}
links None {}
homedetails https://www.zillow.com/homedetails/3395-Michelson-Dr-Irvine-CA-92612/122244533_zpid/ {}
graphsanddata http://www.zillow.com/homedetails/3395-Michelson-Dr-Irvine-CA-92612/122244533_zpid/#charts-and-data {}
mapthishome http://www.zillow.com/homes/122244533_zpid/ {}
comparables http://www.zillow.com/homes/comps/122244533_zpid/ {}
address None {}
street 3395 Michelson Dr {}
zipcode 92612 {}
city Irvine {}
state CA {}
latitude 33.673118 {}
longitude -117.837061 {}
zestimate None {}
amou

['122244533']

> Above address was plugged into for an apartment complex, but returns an individual house. So there's pitfalls in the search API as well.

In [22]:
#https://www.zillow.com/b/villa-coronado-apartment-homes-irvine-ca-5XjT3r/
villa_coronado_apt_irvine = get_search_api('100+Ambazar', '92614')
villa_coronado_apt_irvine

Number of Units Available: 1
{http://www.zillow.com/static/xsd/SearchResults.xsd}searchresults None {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/b20c067/static/xsd/SearchResults.xsd'}
request None {}
address 100 Ambazar {}
citystatezip 92614 {}
message None {}
text Request successfully processed {}
code 0 {}
response None {}
results None {}
result None {}
zpid 2123703954 {}
links None {}
homedetails https://www.zillow.com/homedetails/100-Ambazar-Irvine-CA-92614/2123703954_zpid/ {}
mapthishome http://www.zillow.com/homes/2123703954_zpid/ {}
comparables http://www.zillow.com/homes/comps/2123703954_zpid/ {}
address None {}
street 100 Ambazar {}
zipcode 92614 {}
city Irvine {}
state CA {}
latitude 33.676082 {}
longitude -117.833666 {}
zestimate None {}
amount None {'currency': 'USD'}
last-updated 01/01/1970 {}
oneWeekChange None {'deprecated': 'true'}
valueChange None {}
valuationRange

['2123703954']

> Again the webpage shows availabilities, but the units are not returned.

In [4]:
def get_comps(zpid, deep=False):
    zpid = '2088377383' #Property Zillow ID
    count = '5' #Number of comparison properties to return
    
    if deep:
        url = 'http://www.zillow.com/webservice/GetDeepComps.htm?'
    else:
        url = 'http://www.zillow.com/webservice/GetComps.htm?'
    
    requestURL = url \
               + 'zws-id=' + zws_id \
               + '&zpid=' + zpid \
               + '&count=' + count

    get_comps = r.get(requestURL)
    root_comps = ET.fromstring(get_comps.content)
    
    for child in root_comps.iter('*'):
        print(child.tag, child.text, child.attrib)

In [20]:
get_comps('2088377383')

{http://www.zillow.com/static/xsd/Comps.xsd}comps None {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.zillow.com/static/xsd/Comps.xsd https://www.zillowstatic.com/vstatic/b20c067/static/xsd/Comps.xsd'}
request None {}
zpid 2088377383 {}
count 5 {}
message None {}
text Error: comps not available for the specified property identifier {}
code 503 {}


In [21]:
get_comps('2088894079', deep=True)

{http://www.zillow.com/static/xsd/Comps.xsd}comps None {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.zillow.com/static/xsd/Comps.xsd https://www.zillowstatic.com/vstatic/b20c067/static/xsd/Comps.xsd'}
request None {}
zpid 2088377383 {}
count 5 {}
message None {}
text Error: comps not available for the specified property identifier {}
code 503 {}


## Individual Property Example
---

In [7]:
# https://www.zillow.com/homedetails/424-Vista-Suerte-Newport-Beach-CA-92660/25478746_zpid/?fullpage=true
# zpid 25478746

get_comps('25478746', deep=True)

{http://www.zillow.com/static/xsd/Comps.xsd}comps None {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.zillow.com/static/xsd/Comps.xsd https://www.zillowstatic.com/vstatic/b20c067/static/xsd/Comps.xsd'}
request None {}
zpid 2088377383 {}
count 5 {}
message None {}
text Error: comps not available for the specified property identifier {}
code 503 {}


In [8]:
# https://www.zillow.com/homedetails/2392-Bay-Farm-Pl-Newport-Beach-CA-92660/25476020_zpid/?fullpage=true
get_comps('25476020', deep=True)

{http://www.zillow.com/static/xsd/Comps.xsd}comps None {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.zillow.com/static/xsd/Comps.xsd https://www.zillowstatic.com/vstatic/b20c067/static/xsd/Comps.xsd'}
request None {}
zpid 2088377383 {}
count 5 {}
message None {}
text Error: comps not available for the specified property identifier {}
code 503 {}


> The above properties are rent and sell, respectively. I have tried this API with an example one given by Zillow and it works, but the address was from New York. It seems Zillow's API here has coverage issues too. 
* https://www.zillow.com/zestimate/#acc